SQL INJECTION DETECTION USING GRU AND ONE DIMENSION CNN-GRU 

In [1]:
#%pip install numpy 
#%pip install keras

#%pip install pandas
#%pip install matplotlib

#%pip uninstall tensorflow keras -y
#%pip install tensorflow
#%pip install scikit-learn
#%pip install tensorflow

#%pip install seaborn




Importation of the required libraries

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd
# Visualisation
import matplotlib.pyplot as plt
# Dataset exploring
import os
# Dataset generation
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Transfer learning
from tensorflow.keras.applications import VGG16

# Optimizer
from keras.optimizers import Adam
# Keras layers
from keras.layers import Input, Dense, Dropout, Flatten, AveragePooling2D
# Keras model
from keras.models import Model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


Processing and Extracting the data

In [2]:
import pandas as pd

# Path to your CSV file
#D:/AI model for research
csv_file_path = 'D:/AI model for research/Book1.csv'

# Read the CSV file using the correct delimiter
try:
    df = pd.read_csv(csv_file_path, delimiter=';', quotechar='"', on_bad_lines='skip')

    # Filter the DataFrame for relevant classes only
    relevant_classes = ['benign', 'SQL injection']
    filtered_df = df[df['attack_type'].isin(relevant_classes)]

    # Count instances for each relevant class
    class_counts = filtered_df['attack_type'].value_counts()

    # Prepare the table header
    header = f"{'Class':<20} {'Number of Instances':<20}"
    separator = "-" * len(header)
    output = f"{header}\n{separator}\n"

    # Add the counts to the output
    for attack_type in relevant_classes:
        count = class_counts.get(attack_type, 0)  # Get the count, default to 0 if not present
        output += f"{attack_type:<20} {count:<20}\n"

    # Print the formatted table
    print(output)

except Exception as e:
    print(f"An error occurred: {e}")


Class                Number of Instances 
-----------------------------------------
benign               20867               
SQL injection        20867               



Obtaining data from the csv files to use for the model

In [3]:
TRAIN_PATH = 'D:/AI model for research/Training_dataset.csv'
TEST_PATH = 'D:/AI model for research/Testing_dataset.csv'

Extracting data and training the model

The 1D CNN-GRU Model

Other code with metrics

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, 
                             confusion_matrix, cohen_kappa_score, roc_auc_score, log_loss, precision_recall_fscore_support)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pickle

# Load data
data = pd.read_csv('D:/AI model for research/Full_Dataset_Equal.csv', sep=';')

# Filter only 'benign' and 'SQL Injection'
data = data[data['attack_type'].isin(['benign', 'SQL injection'])]

# Extract features and labels
X = data['payload'].values
y = data['attack_type'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)  # Encodes 'benign' as 0 and 'SQL Injection' as 1

# One-hot encode the labels for model training
y_onehot = to_categorical(y, num_classes=2)

# SQL injection keywords to add to the tokenizer vocabulary
sql_injection_keywords = ['SELECT', 'UNION', 'DROP', 'INSERT', 'DELETE', '--', 'OR', 'AND', 'FROM', 
                          'WHERE', 'TABLE', 'DATABASE', 'ADMIN', 'PASSWORD', 'EXEC', 'CHAR', 
                          'VARCHAR', 'CONCAT', 'SLEEP', 'BENCHMARK']

# Tokenization and padding with OOV handling and added SQL keywords
num_words = 10000  # Set a higher number of words for the tokenizer
tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')

# Manually add SQL injection keywords
tokenizer.fit_on_texts(sql_injection_keywords)

# Fit the tokenizer on the dataset
tokenizer.fit_on_texts(X)

# Save the tokenizer for later use
tokenizer_save_path = 'D:/AI model for research/tokenizer.pkl'
with open(tokenizer_save_path, 'wb') as f:
    pickle.dump(tokenizer, f)
print(f"Tokenizer saved to {tokenizer_save_path}")

# Convert the text to sequences and pad them
max_sequence_length = 100  # Set a fixed max sequence length
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, maxlen=max_sequence_length)

# Save max_sequence_length
max_sequence_length_path = 'D:/AI model for research/max_sequence_length.pkl'
with open(max_sequence_length_path, 'wb') as f:
    pickle.dump(max_sequence_length, f)
print(f"Max sequence length saved to {max_sequence_length_path}")

# Apply MinMaxScaler to scale the padded sequences between 0 and 1
scaler = MinMaxScaler()

# Number of test rounds
num_rounds = 5  # Set the number of rounds

# Storage for metrics across rounds
accuracy_scores, precision_scores, recall_scores, f1_scores, f2_scores = [], [], [], [], []
kappa_scores, auc_scores, log_losses = [], [], []

# Run multiple rounds of training and evaluation
for round_num in range(num_rounds):
    print(f"\nRound {round_num + 1} / {num_rounds}")

    # Split data into train, validation, and test sets
    X_rest, X_test, y_rest_onehot, y_test_onehot = train_test_split(X_padded, y_onehot, test_size=0.1, shuffle=True)

    # Split remaining 90% into 70% training and 20% validation
    X_train, X_val, y_train_onehot, y_val_onehot = train_test_split(X_rest, y_rest_onehot, test_size=0.2222, shuffle=True)

    # Reshape and scale the data
    X_train_flat = X_train.reshape(-1, X_train.shape[1])
    X_val_flat = X_val.reshape(-1, X_val.shape[1])
    X_test_flat = X_test.reshape(-1, X_test.shape[1])

    # Fit and transform
    X_train_scaled = scaler.fit_transform(X_train_flat).reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val_scaled = scaler.transform(X_val_flat).reshape((X_val.shape[0], X_val.shape[1], 1))
    X_test_scaled = scaler.transform(X_test_flat).reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the CNN-GRU model
    from keras.models import Sequential
    from keras.layers import Conv1D, MaxPooling1D, Dropout, GRU, Flatten, Dense
    from keras.optimizers import Adam

    cnn_gru_model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(max_sequence_length, 1)),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        GRU(32, return_sequences=True),
        GRU(32),
        Flatten(),
        Dense(100, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')
    ])
    cnn_gru_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = cnn_gru_model.fit(X_train_scaled, y_train_onehot, epochs=7, validation_data=(X_val_scaled, y_val_onehot), batch_size=32, verbose=0)

    # Predictions on the test set
    y_pred_prob = cnn_gru_model.predict(X_test_scaled)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_test = np.argmax(y_test_onehot, axis=1)

    # Calculate metrics
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred, average='weighted'))
    recall_scores.append(recall_score(y_test, y_pred, average='weighted'))
    f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    f2_scores.append(fbeta_score(y_test, y_pred, beta=2))
    kappa_scores.append(cohen_kappa_score(y_test, y_pred))
    auc_scores.append(roc_auc_score(y_test, y_pred_prob[:, 1]))
    log_losses.append(log_loss(y_test, y_pred_prob))

# Calculate and print mean and std of metrics across all rounds
print("\n--- Final Metrics ---")
print(f"Accuracy: {np.mean(accuracy_scores):.4f} ± {np.std(accuracy_scores):.4f}")
print(f"Precision: {np.mean(precision_scores):.4f} ± {np.std(precision_scores):.4f}")
print(f"Recall: {np.mean(recall_scores):.4f} ± {np.std(recall_scores):.4f}")
print(f"F1 Score: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"F2 Score: {np.mean(f2_scores):.4f} ± {np.std(f2_scores):.4f}")
print(f"Cohen's Kappa: {np.mean(kappa_scores):.4f} ± {np.std(kappa_scores):.4f}")
print(f"ROC-AUC: {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")
print(f"Log-Loss: {np.mean(log_losses):.4f} ± {np.std(log_losses):.4f}")


Tokenizer saved to D:/AI model for research/tokenizer.pkl
Max sequence length saved to D:/AI model for research/max_sequence_length.pkl

Round 1 / 5


c:\Users\ellen\OneDrive\Desktop\ITRI616\Telescope-Simulator\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step

Round 2 / 5


c:\Users\ellen\OneDrive\Desktop\ITRI616\Telescope-Simulator\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step

Round 3 / 5


...

68/68 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step

Round 4 / 5


c:\Users\ellen\OneDrive\Desktop\ITRI616\Telescope-Simulator\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step

Round 5 / 5


c:\Users\ellen\OneDrive\Desktop\ITRI616\Telescope-Simulator\.conda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step

--- Final Metrics ---
Accuracy: 0.9480 ± 0.0028
Precision: 0.9484 ± 0.0032
Recall: 0.9480 ± 0.0028
F1 Score: 0.9480 ± 0.0027
F2 Score: 0.9513 ± 0.0093
Cohen's Kappa: 0.8961 ± 0.0055
ROC-AUC: 0.9903 ± 0.0012
Log-Loss: 0.1299 ± 0.0054


Saving the tokenizer used for training the 1D CNN-GRU model

In [10]:
import pickle

# For CNN-GRU model
#with open('D:/RESEARCH DATASET OF 10000/tokenizer_cnn_gru.pkl', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Save the tokenizer and maxlen together
save_dir = 'D:/AI model for research/'

with open(f'{save_dir}tokenizer_cnn_gru.pkl', 'wb') as handle:
    pickle.dump({'tokenizer': tokenizer, 'maxlen': max_sequence_length}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Saving the 1D CNN-GRU Model

In [11]:
# Save the CNN-GRU model
cnn_gru_model_path = os.path.join('D:/AI model for research/', 'CNN-GRU_SQL_Injection_Detector1.h5')
cnn_gru_model.save(cnn_gru_model_path)
print(f'CNN-GRU model saved to {cnn_gru_model_path}')

CNN-GRU model saved to D:/AI model for research/CNN-GRU_SQL_Injection_Detector1.h5


The GRU Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, 
                             confusion_matrix, cohen_kappa_score, roc_auc_score, log_loss, precision_recall_fscore_support)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import pickle

# Load data
data = pd.read_csv('D:/AI model for research/Full_Dataset_Equal.csv', sep=';')

# Filter only 'benign' and 'SQL Injection'
data = data[data['attack_type'].isin(['benign', 'SQL injection'])]

# Extract features and labels
X = data['payload'].values
y = data['attack_type'].values

# Encode labels
le = LabelEncoder()
y = le.fit_transform(y)  # Encodes 'benign' as 0 and 'SQL Injection' as 1

# One-hot encode the labels for model training
y_onehot = to_categorical(y, num_classes=2)

# Tokenization and padding with OOV handling
num_words = 10000  # Set a higher number of words for the tokenizer
tokenizer = Tokenizer(num_words=num_words, oov_token='<OOV>')  # Set the oov_token for unseen words
tokenizer.fit_on_texts(X)

# Save the tokenizer for later use
tokenizer_save_path = 'D:/AI model for research/tokenizer.pkl'
with open(tokenizer_save_path, 'wb') as f:
    pickle.dump(tokenizer, f)
print(f"Tokenizer saved to {tokenizer_save_path}")

# Convert the text to sequences and pad them
max_sequence_length = 100  # Set a fixed max sequence length
X_tokenized = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_tokenized, maxlen=max_sequence_length)

# Save max_sequence_length
max_sequence_length_path = 'D:/AI model for research/max_sequence_length.pkl'
with open(max_sequence_length_path, 'wb') as f:
    pickle.dump(max_sequence_length, f)
print(f"Max sequence length saved to {max_sequence_length_path}")

# Apply MinMaxScaler to scale the padded sequences between 0 and 1
scaler = MinMaxScaler()

# Split data into train, validation, and test sets
X_rest, X_test, y_rest_onehot, y_test_onehot = train_test_split(X_padded, y_onehot, test_size=0.1, shuffle=False)

# Split remaining 90% into 70% training and 20% validation
X_train, X_val, y_train_onehot, y_val_onehot = train_test_split(X_rest, y_rest_onehot, test_size=0.2222, shuffle=False)

# Reshape the data for scaling
X_train_flat = X_train.reshape(-1, X_train.shape[1])
X_val_flat = X_val.reshape(-1, X_val.shape[1])
X_test_flat = X_test.reshape(-1, X_test.shape[1])

# Fit the scaler on training data and transform
X_train_scaled = scaler.fit_transform(X_train_flat)
X_val_scaled = scaler.transform(X_val_flat)
X_test_scaled = scaler.transform(X_test_flat)

# Reshape back after scaling
X_train_scaled = X_train_scaled.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val_scaled = X_val_scaled.reshape((X_val.shape[0], X_val.shape[1], 1))
X_test_scaled = X_test_scaled.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the GRU model
from keras.models import Sequential
from keras.layers import GRU, Dropout, Flatten, Dense
from keras.optimizers import Adam

gru_model = Sequential([
    GRU(32, return_sequences=True, input_shape=(max_sequence_length, 1)),
    GRU(32),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')
])

# Compile the GRU model
gru_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Measure training time
start_time = time.time()

class_weights = {0: 1.0, 1: 5.0}  # Increase weight for 'SQL Injection'

# Train the model and capture the history
history = gru_model.fit(X_train_scaled, y_train_onehot, 
                        epochs=5, 
                        validation_data=(X_val_scaled, y_val_onehot), 
                        batch_size=16,
                        class_weight=class_weights)

# End training time
training_time = time.time() - start_time
print(f"Training time: {training_time:.2f} seconds")

# Number of parameters
total_params = gru_model.count_params()
print(f"Total number of parameters: {total_params}")

# Predictions on the test set
y_pred_prob = gru_model.predict(X_test_scaled)  # Get the probabilities
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Evaluate model's performance
y_test = np.argmax(y_test_onehot, axis=1)  # Convert one-hot to labels for evaluation
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f2 = precision_recall_fscore_support(y_test, y_pred, beta=2, average='binary')[2]  # F2 score
cohen_kappa = cohen_kappa_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob[:, 1])
logloss = log_loss(y_test, y_pred_prob)

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
TN, FP, FN, TP = conf_matrix.ravel()

# Calculate False Positives, False Negatives
false_positives = FP
false_negatives = FN
true_negatives = TN
true_positives = TP

# Proportion of false positives
proportion_false_positives = false_positives / (false_positives + true_negatives)

# Plot accuracy and loss over epochs
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Compute ROC curve and AUC for the positive class (SQL Injection)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:, 1])
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Print all metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"F2-Score: {f2:.4f}")
print(f"Cohen's Kappa: {cohen_kappa:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print(f"Log-Loss: {logloss:.4f}")
print(f"False Positives: {false_positives}")
print(f"False Negatives: {false_negatives}")
print(f"True Positives: {true_positives}")
print(f"True Negatives: {true_negatives}")
print(f"Proportion of False Positives: {proportion_false_positives:.4f}")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# Save the GRU model
model_save_path = 'D:/AI model for research/gru_model.h5'  # Update this path as needed
gru_model.save(model_save_path)
print(f"Model saved to {model_save_path}")



: 

Saving the tokenizer used by GRU

In [ ]:
import pickle

# For GRU model
#with open('D:/RESEARCH DATASET OF 10000/tokenizer_gru.pkl', 'wb') as handle:
#    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the same for GRU if using different tokenizer or maxlen
with open(f'{save_dir}tokenizer_gru.pkl', 'wb') as handle:
    pickle.dump({'tokenizer': tokenizer, 'maxlen': max_sequence_length}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Saving the  GRU model

In [ ]:
# Create a directory to save the models if it doesn't exist
save_dir = 'D:/AI model for research/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Save the CNN-GRU model
cnn_gru_model_path = os.path.join(save_dir, 'CNN-GRU_SQL_Injection_Detector.h5')
cnn_gru_model.save(cnn_gru_model_path)
print(f'CNN-GRU model saved to {cnn_gru_model_path}')

# Save the GRU model
gru_model_path = os.path.join(save_dir, 'GRU_SQL_Injection_Detector.h5')
gru_model.save(gru_model_path)
print(f'GRU model saved to {gru_model_path}')

CNN-GRU model saved to D:/AI model for research/CNN-GRU_SQL_Injection_Detector.h5
GRU model saved to D:/AI model for research/GRU_SQL_Injection_Detector.h5


Testing the models 

In [5]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd

# Load test data
test_csv_path = 'D:/AI model for research/Testing_dataset.csv'
test_df = pd.read_csv(test_csv_path, delimiter=';')

# Extract features and labels
X_test = test_df['payload'].values
y_test = test_df['attack_type'].values

# Preprocess the labels
le = LabelEncoder()
y_test_encoded = le.fit_transform(y_test)  # Encodes 'benign' as 0 and 'SQL Injection' as 1
y_test_encoded = to_categorical(y_test_encoded, num_classes=2)

# Load the tokenizer and maxlen used for CNN-GRU model
with open('D:/AI model for research/tokenizer_cnn_gru1.pkl', 'rb') as handle:
    cnn_gru_data = pickle.load(handle)
    tokenizer_cnn_gru = cnn_gru_data['tokenizer']
    maxlen_cnn_gru = cnn_gru_data['maxlen']

# Tokenize and pad the payloads for CNN-GRU model
X_test_sequences_cnn_gru = tokenizer_cnn_gru.texts_to_sequences(X_test)
X_test_padded_cnn_gru = pad_sequences(X_test_sequences_cnn_gru, maxlen=maxlen_cnn_gru)

# Load the tokenizer and maxlen used for GRU model
with open('D:/AI model for research/tokenizer_gru.pkl', 'rb') as handle:
    gru_data = pickle.load(handle)
    tokenizer_gru = gru_data['tokenizer']
    maxlen_gru = gru_data['maxlen']

# Tokenize and pad the payloads for GRU model
X_test_sequences_gru = tokenizer_gru.texts_to_sequences(X_test)
X_test_padded_gru = pad_sequences(X_test_sequences_gru, maxlen=maxlen_gru)

# Function to test models
def test_model(model_path, model_name, X_test_padded):
    print(f"\nTesting {model_name} Model:")
    
    # Load the model
    loaded_model = load_model(model_path)
    print(loaded_model.summary())

    # Evaluate the model on the test data
    results = loaded_model.evaluate(X_test_padded, y_test_encoded)
    print('{:<20} {:<20}'.format('Test loss', 'Test accuracy'))
    print('{:<20} {:<20}'.format(np.round(results[0], 2), np.round(results[1], 2)))

    # Predictions and Metrics
    predictions = loaded_model.predict(X_test_padded)
    predicted_classes = np.argmax(predictions, axis=1)

    true_classes = np.argmax(y_test_encoded, axis=1)
    class_labels = ['benign', 'SQL injection']

    report = classification_report(true_classes, predicted_classes, target_names=class_labels)
    print(report)

    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

# Test CNN-GRU Model
cnn_gru_model_path = 'D:/AI model for research/CNN-GRU_SQL_Injection_Detector.h5'
test_model(cnn_gru_model_path, 'CNN-GRU', X_test_padded_cnn_gru)

# Test GRU Model
gru_model_path = 'D:/AI model for research/GRU_SQL_Injection_Detector.h5'
test_model(gru_model_path, 'GRU', X_test_padded_gru)



Testing CNN-GRU Model:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 52, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 26, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 26, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, 26, 64)         │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_3 (GRU)                     │ (None, 64)             │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │         6,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,880 (222.19 KB)

 Trainable params: 56,878 (222.18 KB)

...

 Trainable params: 44,526 (173.93 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
281/283 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9950 - loss: 0.0091

KeyboardInterrupt: 

GUI to test the model

In [13]:
import tkinter as tk
from tkinter import messagebox
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np

# Load the saved CNN-GRU model, tokenizer, scaler, and max_sequence_length
cnn_gru_model_path = 'D:/AI model for research/cnn_gru_sql_injection_model.h5'
tokenizer_path = 'D:/AI model for research/tokenizer.pkl'
scaler_path = 'D:/AI model for research/minmax_scaler.pkl'
max_sequence_length_path = 'D:/AI model for research/max_sequence_length.pkl'

# Load tokenizer, scaler, and max_sequence_length
try:
    with open(tokenizer_path, 'rb') as f:
        tokenizer = pickle.load(f)
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    with open(max_sequence_length_path, 'rb') as f:
        max_sequence_length = pickle.load(f)
    cnn_gru_model = load_model(cnn_gru_model_path, compile=False)
except Exception as e:
    messagebox.showerror("Error", f"Failed to load model or related files: {str(e)}")
    raise e

# Function to test the input SQL query
def test_sql_injection():
    # Get the input from the text box
    user_input = entry.get().strip()
    
    # Validate input
    if not user_input:
        messagebox.showerror("Input Error", "Please enter a valid SQL query.")
        return
    
    # Preprocess the input: Tokenize and pad (OOV token already handled by tokenizer)
    sequences = tokenizer.texts_to_sequences([user_input])
    
    if len(sequences[0]) == 0:
        messagebox.showerror("Input Error", "Invalid input after tokenization. Please try again with a proper SQL query.")
        return

    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
    
    # Scale the padded sequences
    padded_sequences_scaled = scaler.transform(padded_sequences)  # Ensure the sequence length matches training
    padded_sequences_scaled = np.expand_dims(padded_sequences_scaled, axis=2)  # Reshape for the model input
    
    # Predict using the loaded model
    prediction = cnn_gru_model.predict(padded_sequences_scaled)
    predicted_class = np.argmax(prediction, axis=1)[0]  # Extract the single predicted class
    confidence_score = prediction[0][predicted_class] * 100  # Confidence score in percentage
    
    # Display the result with confidence score
    if predicted_class == 0:
        result = f"Benign query with {confidence_score:.2f}% confidence"
    else:
        result = f"SQL Injection (malicious) with {confidence_score:.2f}% confidence"
    
    messagebox.showinfo("Result", result)

# Create the GUI window
root = tk.Tk()
root.title("SQL Injection Detector")

# Create and place the input text box
label = tk.Label(root, text="Enter SQL Query:")
label.pack(pady=10)
entry = tk.Entry(root, width=50)
entry.pack(pady=10)

# Create and place the test button
test_button = tk.Button(root, text="Test Query", command=test_sql_injection)
test_button.pack(pady=20)

# Start the GUI event loop
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 686ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
